In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erfc
from scipy.special import erfcx
import mibitrans as mbt

## Effects of the untruncated Domenico solution

#### Introduction

The analytical equation used in BIOSCREEN uses Domenico (1987) analytical solution for multidimensional transport of a contaminant species, but with the addition of source depletion and source superposition (to allow for more source zones), see equation below;

\begin{align}\tag{1}
    C(x, y, z, t) &= \sum_{i=1}^{n}\left\{ \frac{C^*_{0,i}}{8} \exp \left[-k_s \left(t-\frac{xR}{v} \right)\right] \right. \\
    &\quad \cdot \left( \exp \left[ \frac{x\left(1-P\right)}{2\alpha_x}\right]
    \cdot\operatorname{erfc} \left[ \frac{(x - Pvt)}{2\sqrt{\alpha_x vt }} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y_i/2}{2\sqrt{\alpha_y x}} \right] - \operatorname{erf} \left[ \frac{y - Y_i/2}{2\sqrt{\alpha_y x)}} \right] \right) \\
    &\quad \cdot \left. \left( \operatorname{erf} \left[ \frac{Z}{2\sqrt{\alpha_z x)}} \right] - \operatorname{erf} \left[ \frac{-Z}{2\sqrt{\alpha_z x}} \right] \right) \right\} \\
    & \text{with} \quad P = \sqrt{1+4\mu \alpha_x/v}
\end{align}

However, this solution (implemented as the Bioscreen solution in this package) is a truncation of the full analytical equation, which contains an additional term comprised of a product of an exp and erfc. See equation 2 below;

\begin{align}\tag{2}
    C(x, y, z, t) &= \sum_{i=1}^{n}\left\{ \frac{C^*_{0,i}}{8} \exp \left(-k_s t\right) \right. \\
    &\quad \cdot \left( \exp \left[ \frac{x\left(1-P\right)}{2\alpha_x}\right]
    \cdot \operatorname{erfc} \left[ \frac{x - Pvt}{2\sqrt{\alpha_x vt }} \right] \right.\\
    &\quad \:  + \left.  \exp \left[ \frac{x\left(1+P\right)}{2\alpha_x}\right]
    \cdot \operatorname{erfc} \left[ \frac{x + Pvt}{2\sqrt{\alpha_x vt }} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y_i/2}{2\sqrt{\alpha_y x}} \right] - \operatorname{erf} \left[ \frac{y - Y_i/2}{2\sqrt{\alpha_y x)}} \right] \right) \\
    &\quad \cdot \left. \left( \operatorname{erf} \left[ \frac{Z}{2\sqrt{\alpha_z x)}} \right] - \operatorname{erf} \left[ \frac{-Z}{2\sqrt{\alpha_z x}} \right] \right) \right\} \\
    & \text{with} \quad P = \sqrt{1+4\left(\mu - k_s \right) \alpha_x/v}
\end{align}

As in the current age, the added computing time required for this term is negligible, it is included in the Anatrans model of this package.

This notebook gives an overview of the differences between the truncated and untruncated Domenico solution, and briefly discusses the implementation in the mibitrans package. For ease of explanation, consider a situation with no decay or source depletion, with $\mu = 0$ and $k_s = 0$, thus $P = 1$.

Note that exact analytical models (like Wexler (1992), which is the basis for the mibitrans solution, as implemented in BIOSCREEN-AT) still give more accurate results than the Anatrans or solution, regardless of truncation (West et al., 2007). Furthermore, note the change in the definition of the source depletion term and P in equation 2. This difference is explained and investigated in 'example_source_depletion.ipynb'.


#### Over- and underflow of the additional term

In mibitrans, 64-bit float numpy arrays are used for the calculations, consequently, it only can hold values between approximately $10^{-300}$ and $10^{+300}$. While these are generally beyond any reasonable order of magnitude, certain parameter combinations may cause an underflow (value < $10^{-300}$ and therefore set to 0), or overflow (value > $10^{+300}$ and therefore set to infinity) to occur. Especially at distances further from the source. However, this does not necessarily mean that the additional term is negligibly small, as for $a \to \infty$, $\exp(a) \to \infty$ and $\text{erfc}(a) \to 0$, and $\infty \cdot 0$ is indeterminate. To combat this, the erfcx function is used, where $\text{erfcx}(a) = \exp(a^2) \cdot \operatorname{erfc}(a)$. This transforms the additional term as: $\exp(b) \cdot \text{erfc}(a) = \frac{\exp(b)}{\exp(a^2)} \cdot \exp(a^2) \cdot \operatorname{erfc}(a) = \exp(b - a^2) \cdot \text{erfcx}(a)$. Under any reasonable field conditions, this expression of the additional term will not cause over- or underflow. And is therefore implemented as such in the Anatrans solution.

#### Visualization of the additional term

Consider the following parameters as example field conditions

In [ ]:
hydro = mbt.HydrologicalParameters(
    velocity=0.1,
    porosity=0.3,
    alpha_x=1,
    alpha_y=0,
    alpha_z=0,
)

att = mbt.AttenuationParameters(
    retardation=1,
)

source = mbt.SourceParameters(
    source_zone_boundary=np.array([5,10,15]),
    source_zone_concentration=np.array([15,10,5]),
    total_mass="inf",
    depth=10,
)

model = mbt.ModelParameters(
    model_length=175,
    model_width=50,
    model_time=5*365,
    dx=1,
    dy=5,
    dt=1,
)

In [ ]:
ana_obj = mbt.Anatrans(hydro,att, source, model)
ana_ndeg = ana_obj.run()

ana_ndeg.centerline(time=73, color="green", label="t = 73 days")
ana_ndeg.centerline(time=365, color="blue", label="t = 365 days")
ana_ndeg.centerline(time=3*365, color="red", label="t = 3*365 days")
plt.title("Anatrans concentration distribution at various times")
plt.legend()
plt.show()

Without considering transverse dispersion, source depletion, decay or multiple source zones, equation 1 would look like:

\begin{equation}\tag{3}
    C(x, t) = \frac{C_{0}}{2} \operatorname{erfc} \left( \frac{x - vt}{2\sqrt{\alpha_x vt }}\right)
\end{equation}

And equation 2 would be:

\begin{equation}\tag{4}
    C(x, t) = \frac{C_{0}}{2} \left[ \operatorname{erfc} \left( \frac{x - vt}{2\sqrt{\alpha_x vt }}\right) + \exp \left(\frac{x}{2\alpha_x}\right) \cdot \operatorname{erfc} \left( \frac{x + vt}{2\sqrt{\alpha_x vt }} \right) \right]
\end{equation}

The boundary value for continuous input is $C(x=0, t) = C_0$. However, when evaluated at $x=0$ and $t=0$, the erfc term in equation 3 resolves to $1$ and thus $C(x=0, t=0) = \frac{C_0}{2}$. Therefore, equation 3 and by extension equation 1 do not abide by the boundary condition. This is visualized below.

In [ ]:
xxx = ana_obj.xxx
ttt = ana_obj.ttt
rv = ana_obj.rv
alpha_x = hydro.alpha_x
velocity = hydro.velocity

# Calculate additional term under no decay conditions
erfc_inner = (xxx + rv * ttt) / (2 * np.sqrt(alpha_x * rv * ttt))
add_term = np.exp(xxx / alpha_x - erfc_inner**2) * erfcx(erfc_inner)
# Calculate the advection term
advec_term = erfc((xxx - rv * ttt) / (2 * np.sqrt(alpha_x * rv * ttt)))

# Time does not evaluate at t=0, so add manually for visualization
t = np.zeros(len(ttt[:,0,0])+1)
t[1:] = ttt[:,0,0]
adv_x0 = np.zeros(len(ttt[:,0,0])+1)
add_x0 = np.zeros(len(ttt[:,0,0])+1)
# For x=0 and t=0, terms are equal to 1
adv_x0[0] = 1
add_x0[0] = 1
adv_x0[1:] = advec_term[:,0,0]
add_x0[1:] = add_term[:,0,0]

plt.plot(t, adv_x0, color="blue", label="advective term")
plt.plot(t, add_x0, color="green", label="additional term")
plt.plot(t, adv_x0 + add_x0, linestyle=":", color="black", label="sum")
plt.xlabel("time [days]")
plt.ylabel("value at x=0 [-]")
plt.legend()
plt.show()


Thus, only at around t = 150 days is the advective term is equal to $2$, and the boundary condition at $x=0$ satisfied. When including the additional term, the boundary condition is satisfied at all times. However, the error introduced by the truncation propagates, as shown in the figures below.

In [ ]:
plt.pcolormesh(xxx[None, None, :], ttt[:, None, None], add_term[:, 0, :])
plt.xlabel("x-position [m]")
plt.ylabel("time [days]")
plt.colorbar(label = "value additional term [-]")
plt.show()

x = xxx[0, 0, :]
max_additional_x_term = np.max(add_term[:,0,:], axis = 0)
plt.plot(x, max_additional_x_term, color="black")
plt.xlabel("time [days]")
plt.ylabel("Maximum value additional term [-]")
plt.show()

The value of the additional term is highest at $x=0$ and $t=0$. While the additional term decreases rapidly for higher values of $x$ and $t$, it remains relevant along the advective front. The size of the error that the truncation introduces is dependent on the Peclet number; $Pe = \frac{vx}{D_x}$. And thus for input parameters, dependent on the flow velocity and longitudinal dispersivity.